In [2]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import glob
import math
import xesmf as xe
from scipy.stats.mstats import linregress
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [3]:
T = xr.open_mfdataset('/g/data/rt52/era5/pressure-levels/monthly-averaged/t/*/*.nc')

In [4]:
t=T.t[1:]

In [5]:
level=t.level
lat=t.latitude
time=t.time

height=-7*(np.log(level/1004))
#height.values #height in Km

In [6]:
def take_anom(arg):
    anom=(arg.groupby('time.month'))-(arg.groupby('time.month').mean('time'))
    return(anom)

In [7]:
def linear_regression(x,y):
    slope,intercept,rvalue,pvalue,stderr=linregress(x,y)
    return[slope,rvalue,pvalue,stderr]

In [8]:
t

,Array,Chunk
Bytes,77.29 GB,153.66 MB
Shape,"(503, 37, 721, 1440)","(1, 37, 721, 1440)"
Count,2015 Tasks,503 Chunks
Type,float32,numpy.ndarray


In [9]:
t_out = xr.Dataset({'latitude': (['latitude'], np.arange(-89.5, 89.5, 1.0)),
                    'longitude':(['longitude'], np.arange(0.5, 359.5, 1.0)),
                   }
                  )

regridder = xe.Regridder(t, t_out, 'bilinear')
regridder.clean_weight_file()
regridder

KeyError: 'lon'